In [ ]:
import numpy as np
import pylab as plt
import itertools as it

In [ ]:
# set random-number seed
rng = np.random.default_rng(17)

In [ ]:
# make 3-point point clouds
N = 1_000_000
d = 2
xs = rng.normal(size=(N, 3, d))
xs += 7. * rng.normal(size=(N, d))[:, None, :]

In [ ]:
# compute areas
def area(xs):
    """
    ## comments:
    - Computes the area with the absolute value of a cross product.
    
    ## bugs:
    - Untested; unchecked!
    """
    return 0.5 * np.abs((xs[:, 2, 0] - xs[:, 0, 0]) * # x3 - x1
                        (xs[:, 1, 1] - xs[:, 0, 1]) - # y2 - y1
                        (xs[:, 2, 1] - xs[:, 0, 1]) * # y3 - y1
                        (xs[:, 1, 0] - xs[:, 0, 0]))  # x2 - x1

ys = area(xs) ** 2
print(ys.shape)

In [ ]:
j = 51 # choose one point and plot it
plt.scatter(xs[j, :, 0], xs[j, :, 1])
plt.title("triangle {}: area {}".format(j, np.sqrt(ys[j])))
plt.axis("equal")

In [ ]:
# build polynomial design matrix
def polynomial_terms(xs, max_degree=4):
    """
    ## bugs:
    - `np.concatenate` is slow?
    """
    N, three, d = xs.shape
    assert three == 3
    assert d == 2
    X = [[np.ones(N)]]
    names = ["1"]
    for degree in range(1, max_degree+1):
        thisX = []
        name = []
        for yexp in range(degree+1):
            xexp = degree - yexp
            thisX.append(np.sum(xs[:, :, 0] ** xexp *
                                xs[:, :, 1] ** yexp, axis=1))
            if xexp == 0:
                name.append('(\sum_i y_i^{})'.format(yexp))
            elif yexp == 0:
                name.append('(\sum_i x_i^{})'.format(xexp))
            else:
                name.append('(\sum_i x_i^{}\,y_i^{})'.format(xexp,yexp))
        X.append(thisX)
        names.append(name)
    return X, names

terms, term_names = polynomial_terms(xs)
print(len(terms))
print(list(len(term) for term in terms))
print(list(name for name in term_names))

In [ ]:
def integer_combinations(n, s=None, slist=None):
    if slist is None:
        slist = []
    if s is None:
        ss = 0
        ms = 1
        s = []
    else:
        ss = np.sum(s)
        ms = np.max(s)
    if ss >= n:
        slist += [s]
    for j in range(ms, n - ss + 1):
        slist = integer_combinations(n, s + [j], slist)
    return slist

combinations = integer_combinations(4, None)
print(combinations)

In [ ]:
def count_integers(s, n):
    counts = np.zeros(n+1).astype(int)
    for i in range(n+1):
        counts[i] = np.sum(sum((j == i) for j in s))
    return counts

counts = list(count_integers(combo, 4) for combo in combinations)
print(counts)

In [ ]:
def design_matrix_columns_and_names(terms, term_names, c, n):
    """
    ## bugs:
    - badly repeated code because I suck
    """
    names = []
    columns = []
    foos = it.product(*(it.combinations_with_replacement(term_names[i], c[i]) for i in range(1, n+1)))
    bars = it.product(*(it.combinations_with_replacement(terms[i], c[i]) for i in range(1, n+1)))
    for foo in foos:
        name = ''
        for f in foo:
            if f != ():
                name += '\,'+'\,'.join(f)
        names += [name, ]
    for bar in bars:
        col = 1.
        for f in bar:
            if f != ():
                col *= np.product(np.array(f), axis=0)
        columns += [col, ]
    return columns, names

X = []
column_names = []
for c in counts:
    cs, ns = design_matrix_columns_and_names(terms, term_names, c, 4)
    X += cs
    column_names += ns
X = np.array(X).T
print(len(column_names), column_names)
print(X.shape)

In [ ]:
beta_hat, _, _, _ = np.linalg.lstsq(X, ys, rcond=None)
print(beta_hat)

In [ ]:
print(np.mean(ys))

In [ ]:
print(np.mean((ys - X @ beta_hat) ** 2))

In [ ]:
for j in range(len(beta_hat)):
    if np.abs(beta_hat[j]) > 1e-3:
        print("{:+7.4f}: {}".format(beta_hat[j], column_names[j]))